# Case 2: Pneumonia X-ray image analysis

**Neural Networks for Machine Learning Applications**\
Rabindra Manandhar

14.2.2024 - version 1\
28.2.2024 - version 2

Information Technology, Bachelor's Degree
Metropolia University of Applied Sciences


## 1. Introduction


This notebook creates a binary classifier for x-ray chest images using convolutional neural networks. The main objective is to detect and classify human diseases from the medical images.

The task is to develop a minimum of three different CNN models, calculate the classification reports and confusion matrices for the outcomes and compare their results. The aim is to achieve a minimum of 90% of sensitivity and 90% of specificity in classification results.

## 2. Dataset

The dataset for this case is obtained from the `https://www.kaggle.com/datasets/paultimothymooney/chest-xray-pneumonia/data`.

The dataset is organized into 3 folders (train, test, val) and contains subfolders for each image category (Pneumonia/Normal). There are 5,863 X-Ray images (JPEG) and 2 categories (Pneumonia/Normal).

train => contains the training data/images for teaching our model.

val => contains images which we will use to validate our model. The purpose of this data set is to prevent our model from Overfitting. Overfitting is when your model gets a little too comofortable with the training data and can't handle data it hasn't seen too well.

As the original dataset has only 16 images in the validation folder, the validation dataset is obtained by separating the training dataset into training set/validation set at 80/20. 

test => this contains the data that we use to test the model once it has learned the relationships between the images and their label (Pneumonia/Not-Pneumonia)

In [14]:
import os, shutil

# In the following, insert appropriate path for where the original data is located on your file system,and also the path
# to the destination folder (which is created when executing this).

# Define the path to the original and base directories
original_dir = "/Users/rabindramanandhar/Documents/Metropolia/2024/NeuralNetworksForMachineLearning/Deep-Learning/case2/chest_xray"
base_dir = "/Users/rabindramanandhar/Documents/Metropolia/2024/NeuralNetworksForMachineLearning/Deep-Learning/case2/base"

# Create base directory
os.makedirs(base_dir, exist_ok=True)

In [15]:
# Define the paths to train, test and validation subdirectories

train_dir = os.path.join(base_dir, "train")
test_dir = os.path.join(base_dir, "test")
validation_dir = os.path.join(base_dir, "validation")

# Create the train, test and validation subdirectories
os.makedirs(train_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)
os.makedirs(validation_dir, exist_ok=True)

In [16]:
# Define the paths to normal and pneumonia subdirectories for train, test and validation

train_normal_dir = os.path.join(train_dir, "NORMAL")
train_pneumonia_dir = os.path.join(train_dir, "PNEUMONIA")

test_normal_dir = os.path.join(test_dir, "NORMAL")
test_pneumonia_dir = os.path.join(test_dir, "PNEUMONIA")

validation_normal_dir = os.path.join(validation_dir, "NORMAL")
validation_pneumonia_dir = os.path.join(validation_dir, "PNEUMONIA")

# Create the normal and pneumonia subdirectories for train, test and validation.

os.makedirs(train_normal_dir, exist_ok=True)
os.makedirs(train_pneumonia_dir, exist_ok=True)
os.makedirs(test_normal_dir, exist_ok=True)
os.makedirs(test_pneumonia_dir, exist_ok=True)
os.makedirs(validation_normal_dir, exist_ok=True)
os.makedirs(validation_pneumonia_dir, exist_ok=True)

In [17]:
# Create lists with training file names

# Define paths to original train and test directories
original_train_dir = os.path.join(original_dir, "train")
original_test_dir = os.path.join(original_dir, "test")

original_train_normal_dir = os.path.join(original_train_dir, "NORMAL")
original_train_pneumonia_dir = os.path.join(original_train_dir, "PNEUMONIA")

original_test_normal_dir = os.path.join(original_test_dir, "NORMAL")
original_test_pneumonia_dir = os.path.join(original_test_dir, "PNEUMONIA")

# List files in the normal and pneumonia subdirectories of the original train directory
train_names_normal = os.listdir(original_train_normal_dir)
train_names_pneumonia = os.listdir(original_train_pneumonia_dir)

# List files in the normal and pneumonia subdirectories of the original test directory
test_names_normal = os.listdir(original_test_normal_dir)
test_names_pneumonia = os.listdir(original_test_pneumonia_dir)

print("Train directory:")
print(len(train_names_normal), "normal images")
print(len(train_names_pneumonia), "pneumonia images")

print("Test directory:")
print(len(test_names_normal), "normal images")
print(len(test_names_pneumonia), "pneumonia images")

Train directory:
1349 normal images
3884 pneumonia images
Test directory:
234 normal images
390 pneumonia images


In [18]:
# Shuffle the lists in random order

import random

random.shuffle(train_names_normal)
random.shuffle(train_names_pneumonia)
random.shuffle(test_names_normal)
random.shuffle(test_names_pneumonia)

In [19]:
# Compute splitting points for train/validation/test with 80%/20% to be used below

print("Train data normal:")
print("80%", int(0.8 * len(train_names_normal)))

print("Train data pneumonia:")
print("80%", int(0.8 * len(train_names_pneumonia)))

Train data normal:
80% 1079
Train data pneumonia:
80% 3107


In [20]:
# Copy original training data to new destination base directories in its own subdirectories 80% to train and 20% to validation

# from chest_xray > train > normal to base > train > normal subdirectory
for fname in train_names_normal[:1072]:
    src = os.path.join(original_train_normal_dir, fname)
    dst = os.path.join(train_normal_dir, fname)
    shutil.copyfile(src, dst)

# from chest_xray > train > normal to base > validation > normal subdirectory
for fname in train_names_normal[1072:]:
    src = os.path.join(original_train_normal_dir, fname)
    dst = os.path.join(validation_normal_dir, fname)
    shutil.copyfile(src, dst)

# from chest_xray > train > pneumonia to base > train > pneumonia subdirectory
for fname in train_names_pneumonia[:3100]:
    src = os.path.join(original_train_pneumonia_dir, fname)
    dst = os.path.join(train_pneumonia_dir, fname)
    shutil.copyfile(src, dst)

# from chest_xray > train > pneumonia to base > validation > pneumonia subdirectory
for fname in train_names_pneumonia[3100:]:
    src = os.path.join(original_train_pneumonia_dir, fname)
    dst = os.path.join(validation_pneumonia_dir, fname)
    shutil.copyfile(src, dst)

# Copy original test data to new destinatino base directories in its own subdirectories

# from chest_xray > test > normal to base > test > normal subdirectory
for fname in test_names_normal:
    src = os.path.join(original_test_normal_dir, fname)
    dst = os.path.join(test_normal_dir, fname)
    shutil.copyfile(src, dst)

# from chest_xray > test > pneumonia to base > test > pneumonia subdirectory
for fname in test_names_pneumonia:
    src = os.path.join(original_test_pneumonia_dir, fname)
    dst = os.path.join(test_pneumonia_dir, fname)
    shutil.copyfile(src, dst)

In [21]:
# Check the contents of the new directories

print("Total normal training samples:", len(os.listdir(train_normal_dir)))
print("Total normal validation samples:", len(os.listdir(validation_normal_dir)))
print("Total normal test samples:", len(os.listdir(test_normal_dir)))

print("Total pneumonia training samples:", len(os.listdir(train_pneumonia_dir)))
print("Total pneumonia validation samples:", len(os.listdir(validation_pneumonia_dir)))
print("Total pneumonia test samples:", len(os.listdir(test_pneumonia_dir)))

Total normal training samples: 1349
Total normal validation samples: 1179
Total normal test samples: 234
Total pneumonia training samples: 3884
Total pneumonia validation samples: 3371
Total pneumonia test samples: 390


## 3. Implementation

### 3.1 Setup

The required libraries are as follows:

os - os module allows to access os functionalities and interact with the underlying os that Python is running on.

shutil - for copying files and entire directory trees, as well as for archiving files and directories.

random - to shuffle the images

tensorflow - for building and training neural networks  of various architectures ( CNNs in this case)

matplotlib, seaborn - to visualize/make graphical presentations of the training results

scikit-learn - it provides a wide range of tools for building machine learning models

In [22]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (
    Conv2D,
    MaxPooling2D,
    Flatten,
    Dense,
    BatchNormalization,
    Dropout,
)
from tensorflow.keras import losses
from tensorflow.keras import optimizers
from sklearn.metrics import (
    confusion_matrix,
    classification_report,
    ConfusionMatrixDisplay,
    roc_curve,
    roc_auc_score,
)
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

print("TensorFlow version:", tf.__version__)
print("NumPy version:", np.__version__)

TensorFlow version: 2.18.0
NumPy version: 2.0.2


### Image Preprocessing / Data Augmentation

In order to avoid overfitting problem, we need to expand artificially our dataset. Data augmentation takes the approach of generating more training data from existing training samples by augmenting the samples via a number of random transformations that yield believable-looking images. The goal is that, at training time, the model will never see the exact same picture twice. This helps expose the model to more aspects of the data so it can generalize better.

In Keras, this can be done by adding a number of data augmentation layers at the start of the model.

In [23]:
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    rotation_range=30,
    width_shift_range=0.1,
    height_shift_range=0.1
)

test_datagen = ImageDataGenerator(rescale=1.0 / 255)

Using the flow_from_directory() method from the Keras ImageDataGenerator class to create a data generator for training, validation and test data

In [24]:
train_generator = train_datagen.flow_from_directory(
    train_dir,  # This is the sub-directory in base directory for training images from
    target_size=(150, 150),
    batch_size=32,
    class_mode="binary",
)

validation_generator = test_datagen.flow_from_directory(
    validation_dir, target_size=(150, 150), batch_size=32, class_mode="binary"
)

test_generator = test_datagen.flow_from_directory(
    test_dir, target_size=(150, 150), batch_size=32, class_mode="binary", shuffle=False
)

Found 5232 images belonging to 2 classes.
Found 4549 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


### Building the model

Using Keras with TensorFlow, we build a CNN model with convolutional neural network (CNN) architecture. This architecture consists of convolutional layers followed by max-pooling layers to extract features from the input images. The 'same padding' applies padding to the input images so that the input image gets fully covered by the filter and specified stride. For stride 1, with same padding, the output will be the same as the input. The flattened output is then passed through fully connected layers with dropout regularization to perform classification. Finally, the sigmoid activation function in the output layer provides the probability of the input image belonging to the positive class in a binary classification problem.

In [25]:
# Architecture
model = Sequential(
    [
        Conv2D(
            32, (3, 3), padding="same", activation="relu", input_shape=(150, 150, 3)
        ),
        MaxPooling2D(pool_size=(2, 2)),
        # BatchNormalization(),
        Conv2D(64, (3, 3), padding="same", activation="relu"),
        MaxPooling2D(pool_size=(2, 2)),
        # BatchNormalization(),
        Conv2D(128, (3, 3), padding="same", activation="relu"),
        MaxPooling2D(pool_size=(2, 2)),
        # BatchNormalization(),
        Conv2D(128, (3, 3), padding="same", activation="relu"),
        MaxPooling2D(pool_size=(2, 2)),
        Flatten(),
        Dense(512, activation="relu"),
        Dropout(0.4),
        Dense(1, activation="sigmoid"),
    ]
)

### Compiling

To make the model ready for training, we need to pick three more things as part of the compilation step:

An optimizer — The mechanism through which the model will update the weights of the neural network based on the training data it sees during training , so as to improve its performance i.e. to minimize the loss function.

A loss function — How the model will be able to measure its performance on the training data, and thus how it will be able to steer itself in the right direction.

Metrics - to monitor and evaluate the performance of the model during training and testing — Here, we’ll only care about accuracy (the fraction of the images that were correctly classified).

In [28]:
# Compilation

model.compile(
    optimizer=optimizers.Adam(),
    loss=losses.BinaryCrossentropy(),
    metrics=[tf.metrics.BinaryAccuracy()],
)

model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_4 (Conv2D)               │ (None, 150, 150, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 75, 75, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 75, 75, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 37, 37, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_6 (Conv2D)               │ (None, 37, 37, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_6 (MaxPooling2D)  │ (None, 18, 18, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 18, 18, 128)    │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_7 (MaxPooling2D)  │ (None, 9, 9, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 10368)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 512)            │     5,308,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │           513 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5,550,273 (21.17 MB)

 Trainable params: 5,550,273 (21.17 MB)

 Non-trainable params: 0 (0.00 B)

### Training


In keras, fit() method is used to train the neural network model on a given dataset.

During training, the model will perform the following steps for each epoch:

Forward pass: The model takes the input data (train_generator) and passes it through the neural network to generate predictions.

Computation of loss: The model computes the loss, which is a measure of how well the predictions match the true labels (train_generator.labels).

Backward pass (Backpropagation): The model calculates the gradients of the loss with respect to the model's parameters (weights and biases) using backpropagation.

Optimization: The optimizer adjusts the model's parameters based on the computed gradients to minimize the loss function.

In [ ]:
%%time

epochs = 10 # Increase the number of epochs for better convergence

history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size)

## 4. Performance and Evaluation

### Performance

Instructions:

- Show the training and validation loss and accuracy plots
- Interpret the loss and accuracy plots (e.g. is there under- or over-fitting)
- Describe the final performance of the model with test set

Now that we have trained our model, we check that on average, how good is our model at classifying never-before-seen digits by computing average accuracy over the entire test set using evaluate() method in Keras.

Check what data we have in the h -structure.

In [ ]:
h_dict = history.history
h_dict.keys()

#### Plot the training and validation accuracy

In [ ]:
def plot_accuracy(history):

    plt.plot(history.history["binary_accuracy"], label="train")
    plt.plot(history.history["val_binary_accuracy"], label="validation")
    # plt.ylim([0, 10])
    plt.title("Training accuracy")
    plt.xlabel("Epoch")
    plt.ylabel("Accuracy")
    plt.legend()
    plt.grid(True)

In [ ]:
plot_accuracy(history)
plt.ylim(0.5, 1.05)

#### Plot the training and validation loss

In [ ]:
def plot_loss(history):

    plt.plot(history.history["loss"], label="train")
    plt.plot(history.history["val_loss"], label="validation")
    plt.title("Training loss")
    plt.xlabel("Epoch")
    plt.ylabel("Error")
    plt.legend()
    plt.grid(True)

In [ ]:
plot_loss(history)
plt.ylim(-0.05, 1)

### Evaluation and demonstration of metrics

We calculate the classification report and confusion matrix.

In [ ]:
# Use the test dataset for evaluation
test_loss, test_accuracy = model.evaluate(test_generator)

print(f"Classification Report:")

# Get predicted probabilities
test_pred_prob = model.predict(test_generator)

# Convert probabilities to binary predictions based on a threshold (e.g., 0.5)
test_pred = (test_pred_prob > 0.5).astype(int)
test_labels = test_generator.labels

class_names = ["NORMAL", "PNEUMONIA"]
report = classification_report(test_labels, test_pred, target_names=class_names)
print(report)

Note:

`Precision`: Precision measures the proportion of true positive predictions out of all positive predictions made by the model. It is computed as PPV = TP / (TP + FP)

For class NORMAL, the precision is *, indicating that *% of the predictions for class NORMAL are correct.\
For class PNEUMONIA, the precision is *, meaning only *% of the predictions for class PNEUMONIA are correct.\

`Recall`: Recall (also known as `sensitivity`) measures the rate of true positive cases correctly predicted. It is computed as TPR = TP / (TP + FN).

For class NORMAL, the recall is *, indicating that *% of the actual instances of class NORMAL were correctly predicted by the model.\
For class PNEUMONIA, the recall is *, meaning *% of the actual instances of class PNEUMONIA were correctly predicted by the model.\
F1-score: The F1-score is the harmonic mean of precision and recall. It provides a balance between precision and recall.

For class NORMAL, the F1-score is *, indicating a decent balance between precision and recall for class NORMAL.\
For class PNEUMONIA, the F1-score is *, suggesting that the model struggles to balance precision and recall for class PNEUMONIA.

In [ ]:
# Confusion matrix
print(f"Confusion Matrix:")

conf_matrix = confusion_matrix(test_labels, test_pred)
# print(conf_matrix)

# Plot confusion matrix
disp = ConfusionMatrixDisplay(conf_matrix)
disp.plot()
print(confusion_matrix)
print("True Positives: ", conf_matrix[1, 1])
print("False Negatives: ", conf_matrix[0, 1])
print("False positives: ", conf_matrix[1, 0])
print("True Negatives: ", conf_matrix[0, 0])

# Compute sensitivity, specificity and overall accuracy
tn, fp, fn, tp = conf_matrix.ravel()

sensitivity = tp / (tp + fn)
specificity = tn / (tn + fp)
accuracy = (tp + tn) / (tp + tn + fp + fn)

print("Sensitivity:", sensitivity)
print("Specificity:", specificity)
print("Overall Accuracy:", accuracy)

A confusion matrix is a table used in classification tasks to evaluate the performance of a machine learning model. It presents a summary of the predictions made by the model against the actual labels in a tabular format. Each cell in the matrix represents the count of instances where the model predicted a certain class compared to the true class.

**True Positives (TP):** The model correctly predicted * instances as positive (e.g., correctly identified positive cases).

**False Positives (FP):** The model incorrectly predicted * instances as positive (e.g., falsely identified negative cases as positive).

**True Negatives (TN):** The model correctly predicted * instances as negative (e.g., correctly identified negative cases).

**False Negatives (FN):** The model incorrectly predicted * instances as negative (e.g., falsely identified positive cases as negative).

The above confusion matrix suggests that the model has:

High false positives and low false negatives, indicating that the model is prone to making Type I errors (false alarms).

High true negatives and low true positives, which implies that the model may have difficulty identifying positive instances accurately.


# ROC Curve

In [ ]:
fpr, tpr, thresholds = roc_curve(test_labels, test_pred_prob)
auc = roc_auc_score(test_labels, test_pred_prob)
plt.plot(fpr, tpr, label="Model (auc = {:.3f})".format(auc))
plt.legend(loc=4)
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("Receiver Operating Characteristic (ROC) Curve")
plt.xlim(
    0,
)
plt.ylim(
    0,
)

## 9. Discussion and conclusions

Instructions: Write

- What settings and models were tested before the best model was found
    - What where the results of these experiments
- Summary of  
    - What was your best model and its settings
    - What was the final achieved performance
- What are your main observations and learning points
- Discussion how the model could be improved in future

**Note:** Remember to evaluate the final metrics using the test set.


In this model, we have used CNN's sequential modeling architecture. We used Conv2D with several filters (32, 64, 128) of filter sizes 3x3 followed by Maxpooling2D layer of pool size 2x2. The flattened output is then passed through fully connected layers with dropout regularization to perform classification. Finally, the sigmoid activation function in the output layer provides the probability of the input image belonging to the positive class in a binary classification problem.

The **adam** optimizer with default settings was used.

Totally ~98% of sensitivity ~72% of specificity for the test dataset were achieved with 10 epochs.

The model might be tuned by using more epochs to train the model and then the desired specificity might also be achieved.